# Simple sensitivity study

Author: Raghav Kansal

In [ ]:
from SensitivityStudy import Analyser
import pandas as pd
from pathlib import Path

In [ ]:
# automatically reloads imported files on edits
%load_ext autoreload
%autoreload 2

In [ ]:
MAIN_DIR = Path("/home/users/lumori/bbtautau/")
SIG_KEYS = {"hh": "bbtthh", "he": "bbtthe", "hm": "bbtthm"}  # We should get rid of this

data_dir_2022 = "/ceph/cms/store/user/rkansal/bbtautau/skimmer/25Apr17bbpresel_v12_private_signal"
data_dir_otheryears = "/ceph/cms/store/user/rkansal/bbtautau/skimmer/25Apr24Fix_v12_private_signal"

data_paths = {
    "2022": {
        "data": Path(data_dir_2022),
        "signal": Path(data_dir_2022),
    },
    "2022EE": {
        "data": Path(data_dir_otheryears),
        "signal": Path(data_dir_otheryears),
    },
    "2023": {
        "data": Path(data_dir_otheryears),
        "signal": Path(data_dir_otheryears),
    },
    "2023BPix": {
        "data": Path(data_dir_otheryears),
        "signal": Path(data_dir_otheryears),
    },
}

years = ["2022EE"]  # , "2022EE", "2023", "2023BPix"]
c = "hh"
test_mode = True  # reduces size of data to run all quickly

analyser = Analyser(years, c, test_mode=test_mode)
for year in years:
    analyser.load_year(year)

In [ ]:
analyser.build_tagger_dict()

In [ ]:
analyser.compute_rocs(years)

In [ ]:
analyser.plot_rocs(years, test_mode=test_mode)

In [ ]:
analyser.plot_mass(years, test_mode=test_mode)

In [ ]:
analyser.prepare_sensitivity(years)

In [ ]:
results = {}
for B in [1, 2, 8]:
    yields_B, cuts_B, yields_max_significance, cuts_max_significance = analyser.sig_bkg_opt(
        years, gridsize=30, B=B, plot=True
    )
    sig_yield, bkg_yield = yields_B
    cut_bb, cut_tt = cuts_B
    sig_yield_max_sig, bkg_yield_max_sig = (
        yields_max_significance  # not very clean rn, can be improved but should be the same
    )
    cut_bb_max_sig, cut_tt_max_sig = cuts_max_significance
    results[f"B={B}"] = analyser.as_df(cut_bb, cut_tt, sig_yield, bkg_yield, years)
    print("done with B=", B)

results["Max_significance"] = analyser.as_df(
    cut_bb_max_sig, cut_tt_max_sig, sig_yield_max_sig, bkg_yield_max_sig, years
)
results_df = pd.concat(results, axis=0)
results_df.index = results_df.index.droplevel(1)
print(c, "\n", results_df.T.to_markdown())
results_df.T.to_csv(analyser.plot_dir / f"{'_'.join(years)}-results{'_fast' * test_mode}.csv")